In [3]:
# import libraries
import pandas as pd
import sqlite3

In [4]:
# Assign sqlite database to db variable
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [7]:
# Write a query to only get Nashville providers who refer to, or get referrals from, anybody with an NPI
query = """
SELECT r.*

FROM referrals AS r

JOIN nppes AS n1
    ON r.to_npi = n1.npi

JOIN cbsa AS c
    ON c.ZIP = n1.zip_5

UNION

SELECT r.*

FROM referrals AS r

LEFT JOIN nppes AS n2
    ON r.from_npi = n2.npi

JOIN cbsa AS c
    ON c.ZIP = n2.zip_5
"""

In [8]:
# Create a pandas dataframe with the contents of the query above
nashville_providers = pd.read_sql(query, db)

In [9]:
# Take a look at that dataframe
nashville_providers.head()

,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,335,1023055126,1003013947,69,142,5.535,27.885
1,705,1013154723,1003028770,32,50,41.940,52.320
2,706,1003963976,1003028770,2535,3945,0.000,0.000
3,773,1023055126,1003031261,69,99,37.778,69.205
4,1704,1013109511,1003075862,112,360,23.950,19.562
5,1705,1013124213,1003075862,16,53,22.075,14.147
6,1706,1043220494,1003075862,50,54,21.870,29.621
7,1755,1023046901,1003077728,61,67,20.448,54.516
8,2146,1023055126,1003090424,125,276,4.268,15.862
9,2147,1003863580,1003090424,90,136,14.015,30.774


In [19]:
# Read the dataframe back into a new table, drop the index
nashville_providers.to_sql('nashville_referrals', db, if_exists = 'replace', index = False)

In [11]:
# Check the tables in the database to make sure it's created
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in tables:
    print(table[0])

cbsa
specialty
referrals
nppes
nashville_referrals


In [12]:
# Write a test query against the new table
query = """
SELECT *

FROM nashville_referrals
"""

In [21]:
# Assign the test query results to a dataframe
test = pd.read_sql(query, db)

In [22]:
# Take a look at the top 5 rows of the df
test.head()

,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,335,1023055126,1003013947,69,142,5.535,27.885
1,705,1013154723,1003028770,32,50,41.940,52.320
2,706,1003963976,1003028770,2535,3945,0.000,0.000
3,773,1023055126,1003031261,69,99,37.778,69.205
4,1704,1013109511,1003075862,112,360,23.950,19.562
5,1705,1013124213,1003075862,16,53,22.075,14.147
6,1706,1043220494,1003075862,50,54,21.870,29.621
7,1755,1023046901,1003077728,61,67,20.448,54.516
8,2146,1023055126,1003090424,125,276,4.268,15.862
9,2147,1003863580,1003090424,90,136,14.015,30.774
